In [1]:
import os, numpy as np, pandas, sklearn
import mido, rtmidi, rtmidi_
# np.seed(333)

In [2]:
## NN libs
import keras
from sklearn.decomposition import PCA
from keras.models import Model
from keras.layers import Input, Dense
from keras.utils import to_categorical
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Conv3D, MaxPool2D, Dropout, Flatten
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
# local libs
import config
from data import data, midi

In [4]:
mid = mido.MidiFile(config.dataset_dir + 'mary.mid')

In [5]:
messages = []
# for msg in mid.play(): # play with rests
for msg in mid:
    messages.append(msg)
    
for msg in messages[:4]:
    print(msg)

<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
control_change channel=0 control=91 value=58 time=0
control_change channel=0 control=10 value=69 time=0


In [6]:
msg

<message control_change channel=0 control=10 value=69 time=0>

In [7]:
dirname = config.dataset_dir + 'lmd_matched/A/A/A/'
folder = dirname + os.listdir(dirname)[-1]
filename = folder + '/' + os.listdir(folder)[-1]
mid = mido.MidiFile(filename)
filename

'../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid'

In [8]:
mid

<midi file '../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid' type 1, 14 tracks, 11010 messages>

In [9]:
filename

'../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid'

In [10]:
for x in os.walk(config.dataset_dir + 'lmd_matched/'):
    x = x

In [11]:
fn, _, filelist = x
fn + ' / - ' + filelist[0]

'../datasets/lmd_matched/Z/Z/Z/TRZZZYV128F92E996D / - 1b966417a9aa703873c5fa1cfe18da32.mid'

# Example ml model (nn)

In [12]:
dirname = config.dataset_dir + 'examples/'
filenames = os.listdir(dirname)[:2]
midis = []
for fn in filenames:
    mid = mido.MidiFile(dirname + fn)
    midis.append(mid)

In [13]:
for msg in midis[0]:
    print(msg)

<meta message track_name name='01 16th Snare\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=36 velocity=104 time=0
note_on channel=0 note=38 velocity=57 time=0
note_off channel=0 note=38 velocity=64 time=0.08333333333333333
note_off channel=0 note=36 velocity=64 time=0.020833333333333332
note_on channel=0 note=38 velocity=73 time=0.010416666666666666
note_off channel=0 note=38 velocity=64 time=0.08333333333333333
note_on channel=0 note=38 velocity=107 time=0.041666666666666664
note_off channel=0 note=38 velocity=64 time=0.109375
note_on channel=0 note=38 velocity=104 time=0.020833333333333332
note_off channel=0 note=38 velocity=64 time=0.09895833333333333
note_on channel=0 note=38 velocity=64 time=0.020833333333333332
note_on channel=0 note=36 velocity=103 time=0.00520833333

In [14]:
msg.bytes()

[255, 47, 0]

In [25]:
arrays = [ midi.to_matrix(m) for m in midis ]
arrays = data.make_compatible(arrays)
x_train = np.stack(arrays)
y_train = np.array([[1,0],[0,1]])

is_meta: True | bytes(): [255, 3, 14, 48, 49, 32, 49, 54, 116, 104, 32, 83, 110, 97, 114, 101, 0]
<meta message track_name name='01 16th Snare\x00' time=0>
is_meta: True | bytes(): [255, 88, 4, 4, 2, 36, 8]
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
is_meta: True | bytes(): [255, 88, 4, 4, 2, 36, 8]
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
is_meta: False | bytes(): [144, 36, 104]
note_on channel=0 note=36 velocity=104 time=0
is_meta: False | bytes(): [144, 38, 57]
note_on channel=0 note=38 velocity=57 time=0
is_meta: False | bytes(): [128, 38, 64]
note_off channel=0 note=38 velocity=64 time=0.08333333333333333
is_meta: False | bytes(): [128, 36, 64]
note_off channel=0 note=36 velocity=64 time=0.020833333333333332
is_meta: False | bytes(): [144, 38, 73]
note_on channel=0 note=38 velocity=73 time=0.010416666666666666
is_meta: False | bytes(): [128,

In [17]:
input_shape = x_train.shape
output_shape = y_train.shape
output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
x_train.shape

(2, 56, 3)

In [18]:
y_train.shape

(2, 2)

In [26]:
def sequential_conv(input_shape, output_length, dropout=0.10):
    # Convolutional layers
    
    a = Input(shape=input_shape)
    b = Dense(32)(a)
    model = Model(inputs=a, outputs=b)


#     model = Sequential()
#     model.add(Dense(64, activation='relu', input_shape=input_shape))
#     model.add(Dropout(dropout))
#     model.add(Dense(output_length, activation='softmax'))
    return model, model.summary

dropout = 0.
model, summary = sequential_conv(input_shape, output_length, dropout)
summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 56, 3)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 2, 56, 32)         128       
Total params: 128
Trainable params: 128
Non-trainable params: 0
_________________________________________________________________


In [27]:
learning_rate = 0.01
# sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
optimizer = Adam(lr=learning_rate)
# top_k_categorical_accuracy(y_true, y_pred, k=5)
# https://keras.io/metrics/
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','mean_squared_error'])

In [28]:
batch_size = 8
# n epochs = n iterations over all the training data
epochs = 9

In [29]:
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=1/6)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (2, 56, 3)